# Modules

In [1]:
import os
import numpy as np

from PIL import Image

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

import cv2
import mahotas
import pickle
import cv2
import time

# Utility Functions

In [2]:
def get_image(name, folder):
    filepath = os.path.join(folder, name)
    img = Image.open(filepath)
    return np.array(img)


def get_images(directoryName):
    directory = os.fsencode(directoryName)
    images = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg"):
            image = get_image(filename, directoryName)
            images.append(image)
            continue
        else:
            continue
    return images


def fd_hu_moments(image):
    # Compute the Hu Moments of the image as a feature
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature


def fd_haralick(image):
    # Compute the haralick texture feature vector from the image
    haralick = mahotas.features.haralick(image).mean(axis=0)
    return haralick


def fd_histogram(image):
    # Compute the histogram of the image as a feature
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    cv2.normalize(hist, hist)
    hist = hist.flatten()
    return hist


def getFeatures(image):
    return np.hstack([fd_histogram(image), fd_haralick(image), fd_hu_moments(image)])


# Data Loading & Preprocessing

In [13]:
# Get features for the training images with masks
print("Processing training mask images")
mask_images = get_images("images/mask_train")

mask_features = []
for image in mask_images:
    mask_features.append(getFeatures(image))

scaler = MinMaxScaler(feature_range=(0, 1))
# Normalize The feature vectors...
mask_features = scaler.fit_transform(mask_features)

# Save features to a file
with open('mask_features.pkl', 'wb') as f:
    pickle.dump(mask_features, f)

print("Training Mask Features has shape:", np.shape(mask_features))

# Get features for the training images without masks
print("Processing training no mask images")
nomask_images = get_images("images/unmask_train")

nomask_features = []
for image in nomask_images:
    nomask_features.append(getFeatures(image))

scaler = MinMaxScaler(feature_range=(0, 1))
# Normalize The feature vectors...
nomask_features = scaler.fit_transform(nomask_features)

# Save features to a file
with open('nomask_features.pkl', 'wb') as f:
    pickle.dump(nomask_features, f)

print("Training No Mask Features has shape:", np.shape(nomask_features))

# Get features for test images with mask
print("Processing test mask images")
mask_images_test = get_images("images/mask_test")

mask_features_test = []
for image in mask_images_test:
    mask_features_test.append(getFeatures(image))

scaler = MinMaxScaler(feature_range=(0, 1))
# Normalize The feature vectors...
mask_features_test = scaler.fit_transform(mask_features_test)

# Save features to a file
with open('mask_features_test.pkl', 'wb') as f:
    pickle.dump(mask_features_test, f)
print("Testing Mask Features has shape:", np.shape(mask_features_test))

# Get features for test images without mask
print("Processing test no mask images")
nomask_images_test = get_images("images/unmask_test")

nomask_features_test = []
for image in nomask_images_test:
    nomask_features_test.append(getFeatures(image))

scaler = MinMaxScaler(feature_range=(0, 1))
# Normalize The feature vectors...
nomask_features_test = scaler.fit_transform(nomask_features_test)
print(nomask_features_test)

with open('nomask_features_test.pkl', 'wb') as f:
    pickle.dump(nomask_features_test, f)

print("Testing No Mask Features has shape:",
        np.shape(nomask_features_test))

# Read feature data from file
# with open('mask_features.pkl', 'rb') as f:
#     mask_features = pickle.load(f)
#
# with open('nomask_features.pkl', 'rb') as f:
#     nomask_features = pickle.load(f)
#
# with open('mask_features_test.pkl', 'rb') as f:
#     mask_features_test = pickle.load(f)
#
# with open('nomask_features_test.pkl', 'rb') as f:
#     nomask_features_test = pickle.load(f)


Processing training mask images
Training Mask Features has shape: (617, 276)
Processing training no mask images
Training No Mask Features has shape: (611, 276)
Processing test mask images
Testing Mask Features has shape: (68, 276)
Processing test no mask images
[[0.00130474 0.00361546 0.00976996 ... 0.01912885 0.24567562 0.01654777]
 [0.02008348 0.02529628 0.02734296 ... 0.01913335 0.2438083  0.01655668]
 [0.01738631 0.19512038 0.32872889 ... 1.         1.         1.        ]
 ...
 [0.37978818 0.27524367 0.19688371 ... 0.02113467 0.35472445 0.08365637]
 [0.         0.         0.         ... 0.01999646 0.28194783 0.01633763]
 [0.00416776 0.01332574 0.02280617 ... 0.01909569 0.2420104  0.01649409]]
Testing No Mask Features has shape: (68, 276)


# Model Training

In [4]:
# Read feature data from file
# with open('mask_features.pkl', 'rb') as f:
#     mask_features = pickle.load(f)

# with open('nomask_features.pkl', 'rb') as f:
#     nomask_features = pickle.load(f)

# with open('mask_features_test.pkl', 'rb') as f:
#     mask_features_test = pickle.load(f)

# with open('nomask_features_test.pkl', 'rb') as f:
#     nomask_features_test = pickle.load(f)

# Image labels. Mask = 1.0, No Mask = 0.0
mask_labels = [1.0 for i in mask_features]
nomask_labels = [0.0 for i in nomask_features]
mask_labels_test = [1.0 for i in mask_features_test]
nomask_labels_test = [0.0 for i in mask_features_test]

# Combine mask/no mask features for train and test set
X_train = np.concatenate([mask_features, nomask_features])
y_train = np.concatenate([mask_labels, nomask_labels])

X_test = np.concatenate([mask_features_test, nomask_features_test])
y_test = np.concatenate([mask_labels_test, nomask_labels_test])

# Train the SVM Model
print("Training...")
svm = SVC(kernel='linear', probability=True, random_state=42)
svm.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Model accuracy is: ', accuracy)


Training...
Model accuracy is:  0.9338235294117647


# Testing The Model

In [27]:
size = 4
haar_file = 'haarcascade_frontalface_default.xml'
datasets = 'datasets'

# Part 1: Create fisherRecognizer
print('Recognizing Face Please Be in sufficient Lights...')

# Create a list of images and a list of corresponding names
# (images, labels, names, id) = ([], [], {}, 0)
# for (subdirs, dirs, files) in os.walk(datasets):
#     for subdir in dirs:
#         names[id] = subdir
#         subjectpath = os.path.join(datasets, subdir)
#         for filename in os.listdir(subjectpath):
#             path = subjectpath + '/' + filename
#             label = id
#             images.append(cv2.imread(path, 0))
#             labels.append(int(label))
#         id += 1
(width, height) = (130, 100)

# # Create a Numpy array from the two lists above
# (images, labels) = [numpy.array(lis) for lis in [images, labels]]

# # OpenCV trains a model from the images
# # NOTE FOR OpenCV2: remove '.face'
# model = cv2.face.LBPHFaceRecognizer_create()
# model.train(images, labels)

# Part 2: Use fisherRecognizer on camera stream
face_cascade = cv2.CascadeClassifier(haar_file)
webcam = cv2.VideoCapture(0)
frame_rate = 10
prev = 0
while True:
    time_elapsed = time.time() - prev
    (_, im) = webcam.read()
    if time_elapsed > 1./frame_rate:
        prev = time.time()

        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        input_feature = np.array(getFeatures(
            get_images("images/unmask_test")[0]))
        input_features = np.array(input_feature)
        scaler = MinMaxScaler(feature_range=(0, 1))
        input_features = input_features.reshape(len(input_feature), 1)
        input_features = scaler.fit_transform(
            input_features)
        input_features = input_features.reshape(1, len(input_feature))
        print(input_features)
        y_pred = svm.predict(input_features)
        cv2.imshow('OpenCV', get_images("images/mask_test")[0])
        print(y_pred)
        # faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        # for (x, y, w, h) in faces:
        #     cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
        #     face = gray[y:y + h, x:x + w]
        #     face_resize = cv2.resize(face, (width, height))

        #     input_feature = np.array(getFeatures(
        #         face_resize))
        #     input_features = np.array(input_feature)
        #     scaler = MinMaxScaler(feature_range=(0, 1))
        #     input_features = input_features.reshape(len(input_feature), 1)
        #     input_features = scaler.fit_transform(
        #         input_features)
        #     input_features = input_features.reshape(1, len(input_feature))
        #     print(input_features)
        #     y_pred = svm.predict(input_features)
        #     # Try to recognize the face
        #     # prediction = svm.predict(face_resize)
        #     cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3)

        #     cv2.imshow('OpenCV', im)

        key = cv2.waitKey(10)
        if key == 27:
            break


Recognizing Face Please Be in sufficient Lights...
[[2.25342409e-05 2.25342409e-05 2.26025348e-05 2.26025348e-05
  2.25342409e-05 2.24659471e-05 2.25342409e-05 2.27391226e-05
  2.26708287e-05 2.27391226e-05 2.26025348e-05 2.28757103e-05
  2.27391226e-05 2.30122981e-05 2.26025348e-05 2.28074165e-05
  2.26708287e-05 2.29440042e-05 2.28074165e-05 2.28074165e-05
  2.28757103e-05 2.30805920e-05 2.34220613e-05 2.34220613e-05
  2.36269430e-05 2.36952369e-05 2.39684124e-05 2.34903552e-05
  2.40367063e-05 2.44464695e-05 2.46513511e-05 2.40367063e-05
  2.38318246e-05 2.38318246e-05 2.35586491e-05 2.38318246e-05
  2.34903552e-05 2.34903552e-05 2.39001185e-05 2.41732939e-05
  2.34220613e-05 2.41050000e-05 2.34903552e-05 2.37635308e-05
  2.41732939e-05 2.31488859e-05 2.45830572e-05 2.42415878e-05
  2.40367063e-05 2.38318246e-05 2.41732939e-05 2.36269430e-05
  2.39684124e-05 2.39001185e-05 2.39684124e-05 2.40367063e-05
  2.35586491e-05 2.40367063e-05 2.41732939e-05 2.41732939e-05
  2.41732939e-05 2.

KeyboardInterrupt: 